# PTT 看板(Board) 

### Culture, Consumption, Politics, Gossip.....

In [1]:
'''
Skills：
1.Web Crawler
2.DataPipeLine
3.Dockefile/Docker/DockerHub(port)
4.DataMining(numeric, text mining, image, opinion)
5.Machine Learning/Deep learning
6.Text categorization: Topic mining, Sentiment anaylysis, Opinion mining
'''

'\nSkills：\n1.Web Crawler\n2.DataPipeLine\n3.Dockefile/Docker/DockerHub(port)\n4.DataMining(numeric, text mining, image, opinion)\n5.Machine Learning/Deep learning\n6.Text categorization: Topic mining, Sentiment anaylysis, Opinion mining\n'

# Deploy a MongoDB docker

In [4]:
#把爬取的資料部署裝到容器中, 在Linux中直接執行

In [1]:
!curl -fsSL get.docker.com -o get-docker.sh | sh  #以Linux部署docker容器

In [ ]:
!sudo docker run -p 27017:27017 -v $(pwd)/mongodb_data:/data/db/text --name some-mongo-text-mining -d mongo  
#以port號run一台mongodb的容器,把本地端資料餵進去,輸入password後得到CONTAINER ID, NAME如下：
#CONTAINER ID: 0b6ec25f69cc  
#ONTAINER Names: some-mongo-text-mining

[sudo] password for gueilin2009: 

In [1]:
!pip install pymongo  #載入pymongo連線套件

You are using pip version 9.0.3, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


#  Data Pipeline Design

In [63]:
#功能 (url位址即檔案)
'''
1.將request的網址自動寫入檔案
2.任意設定PTT看板(Board)分類與爬取筆數(Num),每個板約3000多個網頁/每頁約10-20筆文章
3.自動化擷取各檔案的頁面資訊,餵到雲端容器中
4.創BOW(Bag of Words)詞向量,依字頻排序（ex：英文部分）
.....
.....
'''

import requests
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')
import re
!pip install BeautifulSoup4  #Linux packages: BeautifulSoup4
from bs4 import BeautifulSoup   
from collections import Counter 
from pymongo import MongoClient



#取出感興趣的看板中的網頁
def get_ptt(Board, Num):  
    main_page = 'https://www.ptt.cc/bbs/%s'%Board + '/index.html'  
    response = requests.get(main_page)
    total_page = int(response.text.split('a class="btn wide" href="/bbs/%s'%Board +'/index')[2].split('.html">&lsaquo; 上頁</a>')[0]) + 1
    url_template = 'https://www.ptt.cc/bbs/%s'%Board + '/index%s.html'    
    pages_to_crawl = Num  
    for page in range(total_page, total_page - pages_to_crawl,-1):  
        urls = url_template%page  
        return urls
    
    
#取出每個網頁中每筆ptt文章的urls存檔    
def get_ptt_detail(urls, Board, Num):   
    with open('./ptt_data_all/%s'%Num, 'w') as f:   #Linux mkdir ptt_data_all
        html=f.write(requests.get(urls, headers={"cookie":"over18=1"}).text)
    with open('./ptt_data_all/%s'%Num) as f: 
        html=f.read()
        detail_urls_list = re.findall('/bbs/%s'%Board + '/M.{18}html', html)
        return detail_urls_list  


#parse每一則ptt文章的資訊
def parse_ptt_detail(detail_urls_list, Num_detail):
    for i in detail_urls_list:
        with open('./ptt_data_all/details/%s'%Num_detail, 'w') as f:   #linux可直接touch files
            html = f.write(requests.get('https://www.ptt.cc/%s'%i).text)
        with open('./ptt_data_all/details/%s'%Num_detail) as f:
            soup = BeautifulSoup(f.read(), 'html.parser') #Linux using html.parser  Wins using lxml
            metas = [tag.text for tag in soup.find_all('span', {"class":"article-meta-value"})] 
            ptt_data = {}
            ptt_data['author']  = metas[0]
            ptt_data['board'] = metas[1]
            ptt_data['title'] = metas[2]
            ptt_data['dt'] = metas[3]
            content = [tag.text for tag in soup.find_all('div', {"id":"main-content"}, {"class":"bbs-screen bbs-content"})] 
            ptt_data['content'] = content  #推文可再以div.push過濾出來
            return ptt_data

        
#把資料存到雲端容器, 所有code可直接寫成docker file(shell script)
def save_to_mongo(ptt_data):   
    client = MongoClient()
    client.crawler.ptt.insert([ptt_data])  #crawler: database, ptt: collection   #取出塞入的文章id:
    return client


#進入容器, 取出文章text後, 創BOW,或利用R斷詞套件(合併寫在docker file中)
def cnt_words(text, res={})  #arg直接創dict
    bag = text.replace(',','').replace('.','').replace('?','').lower().split()
    for word in bag:
        if word not in stopwords:  #設定stopwords為另一篇ptt文
            if word in res:
                res[word] += 1    #統計wordf freq
            else:
                res[word] = 1    #第一次進來會在這裡
            return res


# Test

In [89]:
#假設設定要爬NBA板10頁

urls = get_ptt('NBA', 10)
detail_urls_list = get_ptt_detail(urls, 'NBA', 10)
ptt_data = parse_ptt_detail(detail_urls_list, 15)  #已爬取
client = save_to_mongo(ptt_data)  
#res = cnt_words(text, res={})   
#res.most_common()   #word freq排序

In [78]:
client  #container key mongo

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [81]:
db = client.crawler
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'crawler')

In [86]:
collection = db.ptt

In [83]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'crawler'), u'ptt')

In [ ]:
collection.get

In [64]:
#進入空器中處理文章,全部完成後打成images上傳dockerhub
# sudo docker exec -it 0b6ec25f69cc  /bin/bash
# root@0b6ec25f69cc:/#